In [40]:
import pandas as pd
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup as BS
import time
import pprint
import json
import requesocks

In [43]:
ls

alameda.csv                oakland.csv                sf_protected_listings.csv
bernal_heights.csv         pacific_heights.csv        south_of_market.csv
daly_city.csv              palo_alto.csv              sunnyvale.csv
menlo_park.csv             potrero_hill.csv           sunset_district.csv
mission.csv                redwood_city.csv           twin_peaks.csv
mission_district_v2.csv    san_bruno.csv
nob_hill.csv               santa_clara.csv


In [45]:
df1 = pd.read_csv('alameda.csv')[['address', 'zid']]
df2 = pd.read_csv('bernal_heights.csv')[['address', 'zid']]
df3 = pd.read_csv('daly_city.csv')[['address', 'zid']]
df4 = pd.read_csv('menlo_park.csv')[['address', 'zid']]
df5 = pd.read_csv('mission.csv')[['address', 'zid']]
df6 = pd.read_csv('mission_district_v2.csv')[['address', 'zid']]
df7 = pd.read_csv('nob_hill.csv')[['address', 'zid']]
df8 = pd.read_csv('oakland.csv')[['address', 'zid']]
df9 = pd.read_csv('pacific_heights.csv')[['address', 'zid']]
df10 = pd.read_csv('palo_alto.csv')[['address', 'zid']]
df11 = pd.read_csv('potrero_hill.csv')[['address', 'zid']]
df12 = pd.read_csv('redwood_city.csv')[['address', 'zid']]
df13 = pd.read_csv('san_bruno.csv')[['address', 'zid']]
df14 = pd.read_csv('santa_clara.csv')[['address', 'zid']]
df15 = pd.read_csv('south_of_market.csv')[['address', 'zid']]
df16 = pd.read_csv('sunnyvale.csv')[['address', 'zid']]
df17 = pd.read_csv('sunset_district.csv')[['address', 'zid']]
df18 = pd.read_csv('twin_peaks.csv')[['address', 'zid']]

In [46]:
df_sf = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8,
                  df9, df10, df11, df12, df13, df14, df15, 
                  df16, df17, df18],
               ignore_index=True,
               axis=0)

In [48]:
df_sf.shape

(1824, 2)

In [49]:
df_sf = df_sf.drop_duplicates()

In [50]:
df_sf.shape

(1737, 2)

In [149]:
# df_sf = df_sf.drop_duplicates()

In [28]:
# df1 = pd.read_csv('ballard.csv')[['address', 'zid']]
# df2 = pd.read_csv('beacon_hill.csv')[['address', 'zid']]
# df3 = pd.read_csv('bellevue_v1.csv')[['address', 'zid']]
# df4 = pd.read_csv('bellevue_v2.csv')[['address', 'zid']]
# df5 = pd.read_csv('burien_v2.csv')[['address', 'zid']]
# df6 = pd.read_csv('cap_hill.csv')[['address', 'zid']]
# df7 = pd.read_csv('issaquah_v1.csv')[['address', 'zid']]
# df8 = pd.read_csv('issaquah_v2.csv')[['address', 'zid']]
# df9 = pd.read_csv('lake_union.csv')[['address', 'zid']]
# df10 = pd.read_csv('magnolia.csv')[['address', 'zid']]
# df11 = pd.read_csv('queen_anne.csv')[['address', 'zid']]

#df12 = pd.read_csv('redmond.csv')[['address', 'zid']]
df13 = pd.read_csv('kirkland_v2.csv')[['address', 'zid']]


In [5]:
df_sea = pd.concat([df12, df13],
                   ignore_index=True,
                   axis=0)

In [11]:
# http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=<ZWSID>&zpid=48749425
ZWSID =  "X1-ZWz1a49jy1eadn_7dhgw"
BASE_URL = "http://www.zillow.com/webservice/GetUpdatedPropertyDetails.htm?zws-id=" + ZWSID + "&zpid="

In [12]:
def get_property_info(zpid):
    return requests.get(BASE_URL + zpid)

In [13]:
# Get some queries
def query_api(df, start_row, stop_row, proxy):
    queries = []
    df = df.ix[start_row : stop_row]
    for row in df.iterrows():
        zpid = str(row[1]['zid'])
        print row[0]
        queries.append(get_property_info(zpid))
    return queries

In [51]:
queries = query_api(df_sf, 0, len(df_sf), False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [74]:
queries[len(queries)-6].content

'<?xml version="1.0" encoding="utf-8"?><UpdatedPropertyDetails:updatedPropertyDetails xmlns:UpdatedPropertyDetails="http://www.zillow.com/static/xsd/UpdatedPropertyDetails.xsd" xsi:schemaLocation="http://www.zillow.com/static/xsd/UpdatedPropertyDetails.xsd http://www.zillowstatic.com/vstatic/eca5e08/static/xsd/UpdatedPropertyDetails.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><request><zpid>19494162</zpid></request><message><text>Request successfully processed</text><code>0</code></message><response><zpid>19494162</zpid><pageViewCount><currentMonth>198</currentMonth><total>198</total></pageViewCount><address><street>1148 La Rochelle Ter UNIT B</street><zipcode>94089</zipcode><city>Sunnyvale</city><state>CA</state><latitude>37.403641</latitude><longitude>-122.011927</longitude></address><links><homeDetails>http://www.zillow.com/homedetails/1148-La-Rochelle-Ter-UNIT-B-Sunnyvale-CA-94089/19494162_zpid/</homeDetails><photoGallery>http://www.zillow.com/homedetails/1148-La-Roc

In [53]:
def get_protected_listings(df, queries):
    protected_listings = []
    for query in queries:
        soup = BS(query.content, 'html.parser')
        if soup.find('text').string == 'Error: protected data is unavailable through API':
            zpid = soup.find('zpid').string
            protected_listings.append(int(zpid))
    return df[df['zid'].isin(protected_listings)]

In [54]:
protected_listings = get_protected_listings(df_sf, queries)

In [65]:
#protected_listings

In [56]:
protected_listings.to_csv('sf_protected_listings_v2.csv', separator=',')

In [121]:
scrape_base_url = "http://www.zillow.com/homes/48749340_zpid/?hdpRedirected=true"

In [57]:
def get_listing_tags(soup):
    response_content = soup.find('response').contents
    attributes = []
    for line in response_content:
        if len(line.contents) > 1:
            for sub_line in line.contents:
                attributes.append((line.name, sub_line.name))
        else:
            attributes.append((line.name))
    return attributes  # returns all of the tags and children tags in a list       

In [58]:
def extract_features(tags, soup):
    schema = {}
    for tag in tags:
        if len(tag) == 2:
            if tag[0] not in schema:
                schema[str(tag[0])] = {}
            schema[str(tag[0])][str(tag[1])] = str(soup.find(tag[0]).find(tag[1]).string)
        if len(tag) > 2:
            try:
                schema[str(tag)] = str(soup.find(tag).string)
            except:
                continue
    return schema # returns a dict with all of the info from the listing, formatted for insertion in to MongoDB

In [79]:
# Now turn all of the html content into a json like structure
# for insertion into MongoDB

def structure_data(queries):
    entries = []
    code_count = 0
    for query in queries: # a query is just a http request response object
            content = query.content
            soup = BS(content, 'html.parser')
            code = int(soup.find('code').string)
            if code == 0:  # a good api call
                tags = get_listing_tags(soup)
                entry = extract_features(tags, soup)
                entries.append(entry)
            
            if code != 0:
                code_count += 1
    print "Could not process {0} queries".format(code_count)
    print "{0} queries successfully processed".format(len(entries))
    return entries

In [80]:
entries = structure_data(queries)

Could not process 1052 queries
604 queries successfully processed


In [66]:
len(queries)

1656

In [1]:
entries[0]

NameError: name 'entries' is not defined

In [61]:
len(entries)

604

In [66]:
# with open('entries_2.json') as f:
#     entries_2 = f.readlines()

In [79]:
entries2 = entries

In [62]:
def insert_queries(entries):
    client = MongoClient()
    db = client['proj']
    listings = db['listings']
    for entry in entries:
        listings.insert(entry)
        #print result.inserted_id

In [63]:
insert_queries(entries)

In [278]:
# write these things to file before I lose them!!!

# with open('entries_2.json', 'w') as j:
#     for line in entries_2:
#         j.write(str(line) + '\n')
#     j.close()

In [53]:
#entries